# GSC - Suchanfragen & Seiten
Bisher haben wir uns rein auf die Gesamtwerte über Zeit beschränkt. Natürlich bieten die Suchanalyse-Daten ebenfalls Suchphrasen und die dazu angezeigten Zielseiten. Für uns SEOs gibt es hierin viele nützliche Informationen.

In [ ]:
%run helpers/code_toggle.py

In [ ]:
from pandas.plotting import register_matplotlib_converters
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dataset

In [ ]:
%load -r 33:38 helpers/helpers.py

## Laden der Daten aus unserer Datenbank in einen DataFrame
Wie schon zuvor, laden wir die Daten, die wir ja nun glücklicherweise in der sqlite Datenbank vorliegen haben mit Pandas direkt in einen DataFrame. Dabei wollen wir nun nicht wie zuvor mittels `pd.read_sql_table` die gesamte Tabelle laden. In diesem Beispiel Fragen wir die Daten mit `pd.read_sql` [(link)](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html#pandas.read_sql) für einen definierten SQL-Query ab. Dabei holen wir die Daten für web, country, device, page & query, die in der entsprechenden Tabelle liegen. 

In [ ]:
db = dataset.connect('sqlite:///data/serienjunkies.db')

query = """
select "date",
    page,
    query,
    sum(clicks) as clicks,
    sum(impressions) as impressions,
    sum(impressions * position) as pos_imp
from web_country_device_page_query
where "date" > '2019-01-01'
group by "date", page, query
"""

df = pd.read_sql(query, con=db.engine, parse_dates=['date'])
df.head()

## Segmentieren nach Suchphrasen und Seiten
Da die GSC Daten sehr umfangreich sind, ist es sehr sinnvoll die Daten zu segmentieren. Dabei schaffen wir weitere Möglichkeiten, den Datenwust zu analysieren.
Dabei werden wir die Suchphrasen in Brand und NonBrand und die Seiten in verschiedene Seitensegmente einteilen.

In [ ]:
%load -r 44:105 helpers/helpers.py

In [ ]:
segments = {
    "brand_patterns": ['serienjunk'],
    "page_patterns" : {
        "serie" : {
            "Serie XYZ" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/$"],
            "Serie XYZ Ausstrahlungstermine TV" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/tv\/"],
            "Serie XYZ News" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/news\/"],
            "Serie XYZ Streams" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/stream\/"],
            "Serie XYZ Poster Übersicht" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/poster\/$"],
            "Serie XYZ Posterseite" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/poster\/.+"],
            "Serie XYZ Episodenguides" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/alle-serien-staffeln\.html"],
            "Serie XYZ Staffel N" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/season\d+\.html"],
            "Serie XYZ Reviewsübersicht" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/reviews\/$"],
            "Serie XYZ Review" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/reviews\/\d{1,3}x\d{1,3}.+\.html"],
            "Serie XYZ Episodenseite" : ["^https:\/\/www\.serienjunkies\.de\/[^\/]+\/\d{1,3}x\d{1,3}.+\.html"]
        }
    }
}

In [ ]:
df = df.pipe(segment_brand, segments) \
    .pipe(segment_pages, segments)

In [ ]:
df.groupby('serie', as_index=False).sum()

## Einfacher Plot der Segmente
Nun da wir die Seiten in Segmente eingeteilt haben, ist es ein leichtes die Leistungsdaten der GSC für diese Segmente darzustellen. Zum einen natürlich als Balkendiagramm, um die Verhältnisse sehen zu können. Zum anderen aber auch Zeitreihen, da wir die Daten inkl. "date" Spalte im DataFrame haben. 

In [ ]:
ax = df.groupby('serie', as_index=False)['clicks'].sum() \
    .sort_values('clicks', ascending=False) \
    .pipe((sns.barplot, 'data'),
          x='clicks', y='serie',
          orient='h')
ax.set_title('Serien-Segmente nach Klicks', x=0, ha='left')
sns.despine()

In [ ]:
sns.set(rc={'figure.constrained_layout.use':False}) # constained layout wird von seaborn aktuell für Grids nicht unterstützt.
df.groupby(['serie','brand_nonbrand'], as_index=False)['clicks'].sum() \
    .sort_values('clicks', ascending=False) \
    .pipe((sns.catplot, 'data'),
          x='clicks', y='serie', col='brand_nonbrand',
          orient='h', kind='bar', aspect=1.5)
sns.despine()

In [ ]:
# %load -r 33:38 helpers/helpers.py
sns.set(context='talk',
        rc={'figure.figsize':(15,6),
            'axes.titlepad':18,
            'axes.titlesize':22,
            'figure.constrained_layout.use':True})

## Segmente als Stripplot
Neben dem gut bekannten Barchart, versuchen wir nun einen etwas cooleren Plot. Im Seaborn-Package, `stripplot` [(link)](https://seaborn.pydata.org/generated/seaborn.stripplot.html#seaborn.stripplot) genannt, zeigt es jede einzelne Suchanfrage (oder Seite), ähnlich wie in einem Scatterplot. In diesem Beispiel zeigen wir zusätzlich die Aufteilung in Brand und Nonbrand. Hiermit erhalten wir einen sehr guten Eindruck davon, welche Bereiche herausstechen.

In [ ]:
ax = df.query('serie != "unknown"') \
    .groupby(['serie','brand_nonbrand', 'page'], as_index=False).sum() \
    .sort_values('clicks', ascending=False) \
    .pipe((sns.stripplot, 'data'), x='clicks', y='serie',
          hue='brand_nonbrand', alpha=.5)
ax.set_title('Seiten pro Segment nach Klicks', x=0, ha='left')
sns.despine()

Natürlich können wir uns die entsprechenden Seiten mit den Maximalwerten auch ausgeben.

In [ ]:
df.query('serie != "unknown"') \
    .groupby(['serie','brand_nonbrand', 'page'], as_index=False)['clicks'].sum() \
    .groupby(['serie', 'brand_nonbrand']).max().unstack(level=1) \
    .sort_values([('clicks','nonbrand')], ascending=False)

In [ ]:
ax = df.query('serie != "unknown"') \
    .groupby(['serie','brand_nonbrand', 'query'], as_index=False).sum() \
    .sort_values('clicks', ascending=False) \
    .pipe((sns.stripplot, 'data'), x='clicks', y='serie',
          hue='brand_nonbrand', alpha=.5)
ax.set_title('Suchphrasen pro Segment nach Klicks', x=0, ha='left')
sns.despine()

Auch hier eine Ausgabe der Top Queries.

In [ ]:
df.query('serie != "unknown"') \
    .groupby(['serie','brand_nonbrand', 'query'], as_index=False)['clicks'].sum() \
    .groupby(['serie', 'brand_nonbrand']).max().unstack(level=1) \
    .sort_values([('clicks','nonbrand')], ascending=False)

### Top Terme eines Segments
Zur Optimierung von Seiten, oder zum "kennenlernen" eines Seitensegments, kann man sich sehr gut die Einzelterme des Datensatzes ansehen. So bekommt man einen recht guten Eindruck davon, welche Einzel-Worte häufig gesucht werden.

In [ ]:
df.query('serie == "Serie XYZ Episodenguides"') \
    .groupby(['query'], as_index=False).sum() \
    ['query'].str.split(expand=True) \
    .melt() \
    .groupby('value').count() \
    .sort_values('variable', ascending=False) \
    .head(10)

## Spaß mit fuzzy wuzzy
Pattern Matching per Distanz-Funktion. Im obrigen Beispiel haben wir die Segmente "nur" auf Basis von Regex-Pattern gebildet. Natürlich gibt es noch wesentlich coolere Methoden. Zum Beispiel können per Levenshtein Distanz ähnliche Suchphrasen gefunden werden. Das Package gibt uns Keywords aus unserem Datenset die eine definierte Ähnlichkeit zum gesuchten Wort aufweisen. 

*Achtung: Wenn du das Package `python-Levenshtein` nicht installieren konntest, gibt es eine Warnung und alles läuft etwas langsamer.*

Das Package führt dabei ein simples Preprocessing durch (lowercase, sorting, etc). Genauere Infos gibt es bei Datacamp https://www.datacamp.com/community/tutorials/fuzzy-string-python

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
fuzz.token_sort_ratio('the walking dead', 'are the dead walking?')

### Klicks mit ähnlicher Suchphrase
Im Beispiel wollen wir nun nur Klicks visualisieren, die durch Suchphrasen generiert wurden, die sehr ähnlich zu unserem gesuchten Begriff sind. Wir suchen im Beispiel nach `the walking dead`.

In [ ]:
unique_queries = df['query'].unique()
len(unique_queries)

In [ ]:
min_score = 90

possibilities = process.extract("the walking dead", unique_queries, limit=100, scorer=fuzz.token_sort_ratio)

hits = pd.DataFrame([possible for possible in possibilities if possible[1] > min_score], columns=['query', 'score']) \
    .set_index('query')

In [ ]:
hits.tail()

Nun joinen wir einfach den DataFrame mit den Suchphrasen die uns "ähnlich" genug sind, an unsere Daten aus der Datenbank. Wir führen dabei einen `right-join` durch. So erhalten wir nur die Zeilen die Suchphrasen aus unserer Treffermenge erhalten.

In [ ]:
ax = df.groupby(['date','query']).sum() \
    .join(hits, on='query', how='right') \
    .groupby(level=0).sum() \
    .plot(y='clicks')
ax.set_title('Klicks für "The Walking Dead"', x=0, ha='left')
sns.despine()

## Top Performer & Low hanging fruits
Als letztes Beispiel machen wir eine klassische Analyse. Wir suchen uns Seiten aus dem Segment `Serie XYZ`, die - bezogen auf CTR vs. Position - schlechter als der Durchschnitt in diesem Seitensegment performen. Damit wir etwas mehr sehen, beschränken wir uns auf Seiten die weniger als 100% CTR haben und eine Position besser als 21 aufweisen. 

### Vorbereiten der Daten
Wir führen nun folgende Schritte aus:
- Filter auf 'nonbrand'
- Filter auf Segment 'Serie XYZ'
- Gruppieren auf Seiten
- Berechnen von CTR und gewichteter Position
- Berechnen von gerundeter Position
- Filter auf Position <= 20 und CTR < 100%

In [ ]:
%load -r 4:28 helpers/helpers.py

In [ ]:
nonbrand_mask = df['brand_nonbrand'].isin(['nonbrand'])
only_serie = df['serie'].isin(['Serie XYZ'])

In [ ]:
df2 = df[nonbrand_mask & only_serie] \
    .groupby('page').sum() \
    .pipe(assign_position) \
    .pipe(assign_ctr) \
    .assign(pos_round = lambda x: round(x['position'], 0)) \
    .query('pos_round <= 20 and ctr < 1')
df2.head()

### Einfacher Stripplot
Zunächst erstellen wir einen einfachen Stripplot. Gerundete Position vs. CTR. Jeder Punkt ist ein Keyword.

In [ ]:
ax = sns.stripplot(x='pos_round', y='ctr', data=df2, alpha=.8)
ax.set_title('Verteilung der CTR pro Position', x=0, ha='left')
sns.despine()

### Ausreißer erkennen
Um nun zu erkennen, welche Keywords besonders (bezogen auf Position und CTR) gut, bzw. weniger gut performen, benötigen wir eine Größe die wir zugrunde legen können. Klassischerweise nutzen wir hierfür den Wert `1.5 x IQR` ([iqr wikipedia](https://en.wikipedia.org/wiki/Interquartile_range)). Beziehungsweise `q1 - 1.5 x IQR` für negative Outlier und `q3 + 1.5 x IQR` für positive Outlier. In unserem Beispiel erachten wir alle Keywords als interessant, die sich unterhalb von Q1 befinden.

In [ ]:
ax = sns.boxplot(x='pos_round', y='ctr', data=df2)
# sns.stripplot(x='pos_round', y='ctr', data=df2, ax=ax, alpha=.4, color='.4', size=5, jitter=True)
ax.set_title('Boxplot für CTR vs. Position', x=0, ha='left')
sns.despine()

### Quantile und IQR berechnen
Um den Datensatz filtern zu können, errechnen wir uns die Quantile, bzw. den IQR pro Positions-Gruppe. So können wir die Seiten klassifizieren und die Daten für weitere Arbeit exportieren, oder hübsch visualisieren.

In [ ]:
groups = df2.groupby('pos_round')
q1 = groups['ctr'].transform(lambda x: x.quantile(.25))
q3 = groups['ctr'].transform(lambda x: x.quantile(.75))
iqr = q3 - q1

In [ ]:
df2['performance'] = 'normal'
df2.loc[df2['ctr'] < q1, 'performance'] = 'low'
df2.loc[df2['ctr'] < q1 - (1.5 * iqr), 'performance'] = 'very-low'
df2.loc[df2['ctr'] > q3, 'performance'] = 'high'
df2.loc[df2['ctr'] > q3 + (1.5 * iqr), 'performance'] = 'very-high'

df2.head()

In [ ]:
ax = sns.stripplot(x='pos_round', y='ctr', data=df2.query('pos_round <= 10'), alpha=.4, size=5, jitter=True,
                   hue='performance', palette={'very-low':'red',
                                               'low':'salmon',
                                               'normal':'midnightblue',
                                               'high':'lightgreen',
                                               'very-high':'green'})
ax.set_title('Performance im Segment "Seite XYZ"', x=0, ha='left')
sns.despine()

### Low-Performer exportieren
Um die Daten zu exportieren, können wir einfach auf die entsprechende Gruppe filtern und mit `.to_csv` zum Beispiel als CSV exportieren.

In [ ]:
df2.query('performance == "low"').sort_values('clicks', ascending=False).head()
# df2.query('performance == "low"').sort_values('clicks', ascending=False).to_csv('low-performer.csv')

Und für alle die sich fragen wie es aussieht, wenn man die Position nicht rundet:

In [ ]:
df3 = df[nonbrand_mask & only_serie] \
    .groupby('page').sum() \
    .pipe(assign_position) \
    .assign(position = lambda x: round(x.position, 1)) \
    .pipe(assign_ctr) \
    .query('position <= 10 and ctr < 1')

groups = df3.groupby('position')
q1 = groups['ctr'].transform(lambda x: x.quantile(.25))
q3 = groups['ctr'].transform(lambda x: x.quantile(.75))
iqr = q3 - q1

df3['performance'] = 'normal'
df3.loc[df3['ctr'] < q1, 'performance'] = 'low'
df3.loc[df3['ctr'] < q1 - (1.5 * iqr), 'performance'] = 'very-low'
df3.loc[df3['ctr'] > q3, 'performance'] = 'high'
df3.loc[df3['ctr'] > q3 + (1.5 * iqr), 'performance'] = 'very-high'

In [ ]:
ax = sns.scatterplot(x='position', y='ctr', data=df3, alpha=.4,
                   hue='performance', palette={'very-low':'red',
                                               'low':'salmon',
                                               'normal':'midnightblue',
                                               'high':'lightgreen',
                                               'very-high':'green'})